In [1]:
import requests
from bs4 import BeautifulSoup
# from selenium import webdriver
from seleniumwire import webdriver
import json, time, random, os, traceback, re, socket, sys
from tqdm import tqdm

In [2]:
CHROMEDRIVER_PATH = "/home/zchen/crawler/chromedriver"

AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
]
REFERERS = [
    "https://en.rti.org.tw",
    "https://google.com",
    "https://YouTube.com",
    "https://Facebook.com",
    "https://twitter.com",
    "https://instagram.com",
    "https://baidu.com",
    "https://wikipedia.org",
    "https://yandex.ru",
    "https://yahoo.com"
]
PROXIES = ['34.82.235.224:3128']

PAGE_ADDRESS_PREFIX = "https://www.cnblogs.com/pick/"
ADDRESS_PREFIX = "https://news.pts.org.tw/article"

In [3]:
ch_options = webdriver.ChromeOptions()

# 不加载图片,加快访问速度
ch_options.add_experimental_option("prefs", {"profile.mamaged_default_content_settings.images": 2})
ch_options.add_argument('--headless') # 无头模式，可不启用界面显示运行
ch_options.add_argument('--disable-gpu') # 禁用GPU加速

# 此步骤很重要，设置为开发者模式，防止被各大网站识别出来使用了Selenium
ch_options.add_experimental_option('excludeSwitches', ['enable-automation'])
# ch_options.add_experimental_option("debuggerAddress", "127.0.0.1:9999")
# ch_options.add_argument('--proxy--server=127.0.0.1:8080')
ch_options.add_argument('--disable-infobars')  # 禁用浏览器正在被自动化程序控制的提示
ch_options.add_argument('--incognito')

# ch_options.add_argument(f'--user-agent="{random.choice(AGENTS)}"') # 设置请求头的User-Agent
ch_options.add_argument(f'--Referer="{random.choice(REFERERS)}"')

browser = webdriver.Chrome(
    options=ch_options,
    # service=webdriver.ChromeService(executable_path=CHROMEDRIVER_PATH)
    )

# Create a request interceptor
def interceptor(request):
    # del request.headers['user-agent']  # Delete the header first
    del request.headers['Referer']
    
    # request.headers['user-agent'] = random.choice(AGENTS)
    request.headers['Referer'] = random.choice(REFERERS)

# Set the interceptor on the driver
browser.request_interceptor = interceptor

In [24]:
def scroll(browser):
    js = '''
        entry_list = document.getElementsByClassName("entry-list list")[0];

        wait = () => new Promise(resolve => {
            window.scrollTo(0, document.body.scrollHeight);
            resolve()
            });

        return (async () => {
            await wait();
            return entry_list.childElementCount
            })();
    '''
    target_len = 150
    while browser.execute_script(js) < target_len:
        time.sleep(0.5)

def request(browser, address, perform_some_actions=None, waiting_time=3):
    browser.get(address)

    # 等待页面 (JavaScript) 加载完成
    time.sleep(waiting_time)

    if perform_some_actions is not None:
        perform_some_actions(browser)

    # 获取页面内容
    content = browser.page_source
    status_code = browser.requests[0].response.status_code
    soup = BeautifulSoup(content, 'html.parser')

    return soup, status_code

In [4]:
def request(browser, address, waiting_time=3):
    headers = {
        'user-agent': random.choice(AGENTS),
        'Referer': random.choice(REFERERS)
    }
    r = requests.get(
        address,
        headers=headers,
        # proxies={'https': proxy}
    )
    soup = BeautifulSoup(r.text, 'html.parser')

    return soup, r.status_code

In [7]:
def crawl(browser, id, waiting_time=3):
    art = {"id": id}
    soup, status_code = request(
        browser,
        f"{ADDRESS_PREFIX}/{id}"
        )
    
    # extract the info
    info = soup.find('script', type="application/ld+json")
    if info is None:
        print("info is None")
        return None, status_code
    art["metadata"] = info.text

    # extract the headline
    headline = soup.find('h1', class_="article-title")
    if headline is None:
        print("headline is None")
        return None, status_code
    art["headline"] = headline.text

    # extract the text
    paras = soup.find('div', class_="post-article")
    if paras is None:
        print("paras is None")
        return None, status_code
    art["article_part"] = str(paras)
    
    return art, status_code

In [12]:
def extract_zhihu_links(soup):
    # print(soup.prettify())
    title_tag_list = soup.find_all(class_="ContentItem-title")
    return [title_tag.find("a").get("href") for title_tag in title_tag_list]

def extract_juejin_links(soup):
    # print(soup.prettify())
    title_tag_list = soup.find_all(class_="title-row")
    return [title_tag.find("a").get("href") for title_tag in title_tag_list]

def extract_links(browser, link, extract, perform_some_actions=None):
    soup, status_code = request(
        browser,
        link,
        perform_some_actions
        )

    # extract the IDs
    return extract(soup)

In [25]:
links = extract_links(
    browser,
    link="https://juejin.cn/recommended?sort=newest",
    extract=extract_juejin_links,
    perform_some_actions=scroll,
    )
print(len(links))
print(links)

158
['/post/7316349124599808036', '/post/7316357622847963155', '/post/7316354375700447270', '/post/7316202809384468489', '/post/7316565794149924879', '/post/7316349124599758884', '/post/7316357622848208915', '/post/7316357622847881235', '/post/7316171289097945122', '/post/7316399461558091813', '/post/7316447817161572387', '/post/7316430287842623514', '/post/7316409548994625574', '/post/7316354375701168166', '/post/7316409548994527270', '/post/7316354375701151782', '/post/7316203818651435017', '/post/7316374543861121058', '/post/7316357622848126995', '/post/7316347993790595108', '/post/7316321509857263656', '/post/7316354375700987942', '/post/7316357622847930387', '/post/7316357622847782931', '/post/7316097536548552743', '/post/7316382839220748288', '/post/7316374543860760610', '/post/7316321509857034280', '/post/7316415580101525558', '/post/7316451458840592419', '/post/7316447817161392163', '/post/7316539465022144512', '/post/7316450414157053967', '/post/7316451458839887907', '/post/73

In [6]:
art, status = crawl(browser, 629729)
print(art)
print()
print("status:")
print(status)

NameError: name 'crawl' is not defined

In [4]:
soup = BeautifulSoup("", 'html.parser')
print(soup.find_all("a"))

[]


In [5]:
for ind, i in tqdm(list(enumerate(range(10)))):
    print(i)

100%|██████████| 10/10 [00:00<00:00, 49578.06it/s]

0
1
2
3
4
5
6
7
8
9


In [1]:
x = None
x.find()

AttributeError: 'NoneType' object has no attribute 'find'